In [ ]:
import os
from dotenv import load_dotenv

import pandas as pd
import psycopg2 as pg
from sqlalchemy import create_engine
import panel as pn

pn.extension('tabulator')

# =========================
# CONEXAO COM O BANCO
# =========================
load_dotenv()

DB_HOST = os.getenv('DB_HOST')
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASS = os.getenv('DB_PASS')
DB_PORT = os.getenv('DB_PORT', '5432')

con = pg.connect(
    host=DB_HOST,
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASS,
    port=DB_PORT
)

engine = create_engine(
    f'postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
)

# =========================
# WIDGETS
# =========================

# SERIAL no banco (nao entra no INSERT)
id_visita = pn.widgets.IntInput(
    name="ID da Visita",
    value=None,
    start=1
)

data_ = pn.widgets.DatePicker(name="Data")

horario_entrada = pn.widgets.TimePicker(
    name="Horario de Entrada",
    value=None
)

horario_saida = pn.widgets.TimePicker(
    name="Horario de Saida",
    value=None
)

num_visitantes = pn.widgets.IntInput(
    name="Visitantes",
    value=1,
    start=1
)

buttonConsultar = pn.widgets.Button(name="Consultar", button_type="primary")
buttonInserir = pn.widgets.Button(name="Inserir", button_type="success")
buttonAtualizar = pn.widgets.Button(name="Atualizar", button_type="warning")
buttonExcluir = pn.widgets.Button(name="Excluir", button_type="danger")

output = pn.Column()

# =========================
# SQL BASE (HORARIOS FORMATADOS)
# =========================

SQL_BASE = """
select
    id_visita,
    data_,
    to_char(horario_entrada, 'HH24:MI') as horario_entrada,
    to_char(horario_saida, 'HH24:MI') as horario_saida,
    num_visitantes
from public.visita_cultural
"""

# =========================
# FUNCOES
# =========================

def carregar_tabela():
    df = pd.read_sql(SQL_BASE, engine)
    return pn.widgets.Tabulator(df, pagination="remote", page_size=10)

def consultar(event):
    if id_visita.value:
        sql = SQL_BASE + " where id_visita = %s"
        df = pd.read_sql(sql, engine, params=(id_visita.value,))
    else:
        df = pd.read_sql(SQL_BASE, engine)

    output.clear()
    output.append(pn.widgets.Tabulator(df))

def inserir(event):
    try:
        with con.cursor() as cursor:
            cursor.execute(
                """
                insert into public.visita_cultural
                (data_, horario_entrada, horario_saida, num_visitantes)
                values (%s, %s, %s, %s)
                """,
                (
                    data_.value,
                    horario_entrada.value,
                    horario_saida.value,
                    num_visitantes.value
                )
            )
            con.commit()

        output.clear()
        output.append(pn.pane.Alert("Inserido com sucesso", alert_type="success"))
        output.append(carregar_tabela())

    except Exception as e:
        con.rollback()
        output.clear()
        output.append(pn.pane.Alert(str(e), alert_type="danger"))

def atualizar(event):
    if not id_visita.value:
        output.clear()
        output.append(
            pn.pane.Alert("Informe o ID da visita para atualizar", alert_type="warning")
        )
        return

    try:
        with con.cursor() as cursor:
            cursor.execute(
                """
                update public.visita_cultural
                set
                    data_ = %s,
                    horario_entrada = %s,
                    horario_saida = %s,
                    num_visitantes = %s
                where id_visita = %s
                """,
                (
                    data_.value,
                    horario_entrada.value,
                    horario_saida.value,
                    num_visitantes.value,
                    id_visita.value
                )
            )
            con.commit()

        output.clear()
        output.append(pn.pane.Alert("Atualizado com sucesso", alert_type="success"))
        output.append(carregar_tabela())

    except Exception as e:
        con.rollback()
        output.clear()
        output.append(pn.pane.Alert(str(e), alert_type="danger"))

def excluir(event):
    if not id_visita.value:
        output.clear()
        output.append(
            pn.pane.Alert("Informe o ID da visita para excluir", alert_type="warning")
        )
        return

    try:
        with con.cursor() as cursor:
            cursor.execute(
                "delete from public.visita_cultural where id_visita = %s",
                (id_visita.value,)
            )
            con.commit()

        output.clear()
        output.append(pn.pane.Alert("Excluido com sucesso", alert_type="success"))
        output.append(carregar_tabela())

    except Exception as e:
        con.rollback()
        output.clear()
        output.append(pn.pane.Alert(str(e), alert_type="danger"))

# =========================
# CALLBACKS
# =========================

buttonConsultar.on_click(consultar)
buttonInserir.on_click(inserir)
buttonAtualizar.on_click(atualizar)
buttonExcluir.on_click(excluir)

# =========================
# LAYOUT
# =========================

pn.Row(
    pn.Column(
        "## Visita Cultural CRUD",
        id_visita,
        data_,
        horario_entrada,
        horario_saida,
        num_visitantes,
        pn.Row(buttonConsultar, buttonInserir),
        pn.Row(buttonAtualizar, buttonExcluir),
    ),
    output
).servable()
